In [ ]:
! pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /tmp/pip-req-build-h_3e_ll3
  Running command git clone -q git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /tmp/pip-req-build-h_3e_ll3
     |████████████████████████████████| 5.6 MB 6.2 MB/s 
     |████████████████████████████████| 117 kB 14.7 MB/s 
     |████████████████████████████████| 181 kB 9.5 MB/s 
     |████████████████████████████████| 76 kB 2.7 MB/s 
     |████████████████████████████████| 132 kB 40.1 MB/s 
     |████████████████████████████████| 32.8 MB 91 kB/s 
     |████████████████████████████████| 8.6 MB 36.7 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 138 kB 46.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 5.1 MB 47.7 MB/s 
     |████████████████████████████████| 127 kB 49.0 MB/s 
  Created wheel for lucem-illud: filename=lucem_illud-8.0.1-py3-non

In [ ]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import scipy #For frequency analysis
import scipy.fftpack
import nltk #the Natural Language Toolkit
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer
import IPython #To show stuff

#Image handling install as Pillow
import PIL
import PIL.ImageOps

#install as scikit-image, this does the image manupulation
import skimage
import skimage.feature
import skimage.segmentation
import skimage.filters
import skimage.color
import skimage.graph
import skimage.future.graph

#these three do audio handling
import pydub #Requires ffmpeg to be installed https://www.ffmpeg.org/download.html; on a mac "brew install ffmpeg"
import speech_recognition #install as speechrecognition
import soundfile #Install as pysoundfile 

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning it may generate.
%matplotlib inline

import os
import os.path
import csv
import re

In [ ]:
from torchvision import models
import torch
from PIL import Image
from torchvision import transforms
import torchvision.transforms as transforms
import pandas as pd

In [ ]:
model = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
#This is code that will transform images to the appropriate format
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
    
#organize class data so it can be mapped effectively
# classes = classes[4:len(classes)]
labels = [''.join([i for i in x if not i.isdigit()]) for x in classes]
labels = [x.replace(', ',' ') for x in labels]

In [ ]:
len(labels)

1000

In [ ]:
path = "Exotic meats.jpg"

In [ ]:
img = Image.open(path).convert('RGB')
img_t = transformations(img)
batch_t = torch.unsqueeze(img_t, 0)
prediction = model(batch_t)

_, indices = torch.sort(prediction, descending=True)
percentage = torch.nn.functional.softmax(prediction, dim=1)[0] * 100
percentages_raw = [percentage[x].item() for x in range(0, len(percentage))] 

top10 = [(labels[idx], percentage[idx].item()) for idx in indices[0][:10]]

In [ ]:
top10

[('digital clock', 10.997919082641602),
 ('street sign', 10.181918144226074),
 ('stopwatch stop watch', 6.561995506286621),
 ('pick plectrum plectron', 4.263008117675781),
 ('book jacket dust cover dust jacket dust wrapper', 3.6164467334747314),
 ('scoreboard', 2.6933019161224365),
 ('coffee mug', 2.353563070297241),
 ('hair spray', 2.338160276412964),
 ('digital watch', 2.251467227935791),
 ('analog clock', 1.7816765308380127)]

In [ ]:
# Load the pretrained model
model = models.resnet18(pretrained=True)

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
d = model.eval()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
def get_vector(image):
    # code adapted from the package:
    # https://github.com/christiansafka/img2vec/
    # Create a PyTorch tensor with the transformed image
    t_img = transformations(image)
    # Create a vector of zeros that will hold our feature vector
    # The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)

    # Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.flatten())                 # <-- flatten

    # Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # Run the model on our transformed image
    with torch.no_grad():                               # <-- no_grad context
        model(t_img.unsqueeze(0))                       # <-- unsqueeze
    # Detach our copy function from the layer
    h.remove()
    # Return the feature vector
    return my_embedding

In [ ]:
from PIL import Image

In [ ]:
image = Image.open(path).convert('RGB')

In [ ]:
get_vector(image)

tensor([1.2961e-01, 5.0820e-01, 1.0357e+00, 4.9727e-01, 9.4013e-02, 2.5376e-02,
        2.6138e-01, 4.1065e-02, 1.5600e-01, 7.4006e-01, 1.7867e-01, 0.0000e+00,
        1.6275e-01, 3.9188e-01, 9.1892e-02, 1.9594e-01, 4.9490e-01, 1.2724e+00,
        1.1884e-01, 1.1242e+00, 6.4183e-03, 4.4351e-03, 0.0000e+00, 9.6133e-01,
        3.2996e-01, 2.1544e-01, 5.4920e-01, 2.3968e-01, 2.7794e-02, 5.0879e-02,
        5.2233e-01, 1.6059e-01, 0.0000e+00, 3.4569e+00, 7.1288e-02, 2.4813e-01,
        0.0000e+00, 1.8240e-01, 4.7743e-01, 7.6064e-01, 1.2297e-01, 3.5354e-02,
        9.3331e-03, 1.5521e-03, 1.1357e-01, 1.6808e-01, 2.3730e-01, 0.0000e+00,
        5.1510e-02, 6.5469e-01, 7.7059e-02, 1.8208e-01, 2.0347e-01, 0.0000e+00,
        1.9134e-01, 5.2209e-02, 4.1137e-03, 8.2258e-01, 4.8536e-03, 2.2255e-01,
        7.6306e-02, 2.8299e-02, 3.0399e-01, 4.9006e-01, 8.8697e-02, 5.8224e-01,
        4.1175e-01, 3.2266e-02, 3.3861e-02, 7.9374e-02, 2.6085e-01, 1.8465e+00,
        1.7842e-01, 1.6052e-01, 5.0917e-

In [ ]:
! pip install img2vec_pytorch

In [ ]:
from img2vec_pytorch import Img2Vec # pip install img2vec_pytorch

In [ ]:
img2vec = Img2Vec()

In [ ]:
flower_vec = img2vec.get_vec(image)
flower_vec

array([1.78246871e-02, 3.58471960e-01, 1.85554719e+00, 1.01652777e+00,
       6.61970768e-03, 4.71305475e-03, 5.44069946e-01, 1.56616338e-03,
       1.09112792e-01, 7.23984063e-01, 9.06784013e-02, 2.16329899e-02,
       2.55619343e-02, 1.23133726e-01, 2.59444714e-02, 1.89316630e-01,
       5.65653965e-02, 1.05651605e+00, 1.02514148e-01, 3.71927381e-01,
       1.39463262e-03, 3.46817523e-01, 6.31888033e-05, 1.68728530e+00,
       4.07781959e-01, 3.92463148e-01, 5.93387067e-01, 3.38585764e-01,
       0.00000000e+00, 8.63548648e-03, 3.67227077e-01, 2.65728449e-03,
       0.00000000e+00, 3.40272617e+00, 1.78601503e-01, 5.13932884e-01,
       8.34475178e-03, 1.06619867e-02, 5.88908076e-01, 1.09031463e+00,
       5.35516143e-02, 3.84999700e-02, 1.82890400e-01, 5.23406500e-03,
       3.84181827e-01, 2.09920168e-01, 3.78145576e-01, 0.00000000e+00,
       1.14161558e-01, 7.42576003e-01, 1.81001306e-01, 3.71466160e-01,
       1.96224228e-02, 1.74850726e-03, 4.78483468e-01, 1.12826638e-01,
      

In [ ]:
cat_image = Image.open("Do Only Good Pet Food.jpg")

In [ ]:
dog_image = Image.open("Crumb Hunter.jpg")

In [ ]:
cat_vec = img2vec.get_vec(cat_image.convert('RGB'), tensor=True)
dog_vec = img2vec.get_vec(dog_image.convert('RGB'), tensor=True)

In [ ]:
flower_vec = img2vec.get_vec(image, tensor=True)

In [ ]:
import torch.nn as nn
import torch

In [ ]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(cat_vec,
              dog_vec)

In [ ]:
cos_sim

tensor([[[0.5562]]])

In [ ]:
cos_sim = cos(flower_vec,
              dog_vec)
cos_sim

tensor([[[0.5141]]])

In [ ]:
path = 'Do Only Good Pet Food.jpg'
img = Image.open(path).convert('RGB')
img_t = transformations(img)
batch_t = torch.unsqueeze(img_t, 0)
prediction = model(batch_t)

_, indices = torch.sort(prediction, descending=True)
percentage = torch.nn.functional.softmax(prediction, dim=1)[0] * 100
percentages_raw = [percentage[x].item() for x in range(0, len(percentage))] 

top10 = [(labels[idx], percentage[idx].item()) for idx in indices[0][:10]]
top10

[('golden retriever', 78.28560638427734),
 ('Labrador retriever', 4.666506767272949),
 ('cocker spaniel English cocker spaniel cocker', 2.836970567703247),
 ('clumber clumber spaniel', 2.6449553966522217),
 ('Saluki gazelle hound', 2.2587971687316895),
 ('English setter', 1.6274504661560059),
 ('kuvasz', 1.436903476715088),
 ('Brittany spaniel', 1.3256767988204956),
 ('Great Pyrenees', 1.223172903060913),
 ('beagle', 0.850166380405426)]